# Data analysis on "Immobiliare.it" data.
After some feature engineering and exploratory analysis, different models will be tried to identify the logic that will best fit the task.

## Feature exploration
The data will be loaded and the features explored.

In [49]:
# Importing the necessary libraries
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as plt
import re

In [2]:
houses = pd.read_csv("DATA/casemilano.csv", encoding='unicode_escape')

In [3]:
#Missing data will be removed
#This leads to lose about 35% of observations, but it must be done, as the variables with missing values are important ones, and cannot be excluded.
houses.dropna(inplace=True)

In [4]:
houses.shape

(7297, 18)

In [5]:
#Functions used to clean observations.

#This function is not important
def lotShared(x):
    return x in sharedALot

#THis functions replaces "pattern" with "sub".
def removePattern(x, pattern, sub):
    return str(x).replace(pattern, sub)
    


### Feature cleaning

The following chunks will be use for feature cleaning and engineering.

In [6]:
# The features will be cleaned.

#Features eliminated
houses = houses.drop('w', axis=1) #It's a constant variable
houses = houses.drop("s", axis=1) #It's unclear the meaning of the variable

#Ascensore recoded.
houses.loc[houses["ascensore"] == "no", "ascensore"] = 0
houses.loc[houses["ascensore"] != 0, "ascensore"] = 1
houses["ascensore"] = houses["ascensore"].astype(np.int8)

#Parking: more than 2 in shared parking space are merged in "aLotShared".
sharedALot = houses["parcheggio"].value_counts().loc[houses["parcheggio"].value_counts() <=5].index
houses.loc[houses["parcheggio"].apply(lotShared),"parcheggio"] = "aLotShared"
houses["parcheggio"] = houses["parcheggio"].astype(str)

#Features made character. The dtype remain "object"
houses["numero.bagni"] = houses["numero.bagni"].astype(str)
houses["numero.stanze"] = houses["numero.stanze"].astype(str)

#Features made numeric
houses.loc[houses["numero.totale.piani.edificio"] == "1 piano", "numero.totale.piani.edificio"] = 1
houses["numero.totale.piani.edificio"] = houses["numero.totale.piani.edificio"].astype(np.int8)

The following are specific changes made to the variable "disponibile". The new version is measured in "days from now", and is a measure of when, in the future, the house will be available.

In [7]:
#The variable disponibile has been converted to "days from the first one".
houses.loc[:,"disponibile"] = houses["disponibile"].apply(removePattern, args = ("disponibile dal ",""))
houses.loc[:,"disponibile"] = houses["disponibile"].apply(removePattern, args = ("disponibile", ""))

In [8]:
#Houses with disponibilità > 0 become datetime tipe.
houses.loc[houses["disponibile"] != "","disponibile"] = pd.to_datetime(houses.loc[houses["disponibile"] != "","disponibile"], format = "%d/%m/%Y")

In [9]:
#Determine the strating point (smallest date - 1 day).
start_time = houses.loc[houses["disponibile"] != "","disponibile"].min() - dt.timedelta(days=1)

In [10]:
#Assegnare ai "disponibili" il valore iniziale
houses.loc[houses["disponibile"] == "","disponibile"] = start_time

In [11]:
#Rendere a series di dtype "date" e togliere il valore iniziale.
houses["disponibile"] = pd.to_datetime(houses["disponibile"])

In [12]:
#Togliere il giorno iniziale
houses["disponibile"] = pd.to_numeric(houses["disponibile"] - start_time)

In [13]:
houses.head()

,prezzo,metri.quadrati,numero.bagni,ascensore,numero.stanze,altre.caratteristiche,numero.totale.piani.edificio,parcheggio,disponibile,spese.condominiali,anno.di. Costruzione,condizioni,zona,piano,riscaldamento.centralizzato,classe.di.efficienza.energetica
1,500000.0,57.0,1,1,2,fibra ottica | videocitofono | sistema d'allar...,7,1 in garage/box,0,nessun costo condominiale,2010.0,eccellente / restaurato,porta romana - medaglie d'oro,1,centralizzato,e
2,395000.0,92.0,1,1,3,cancello elettrico | fibra ottica | videocitof...,8,no,0,200,1960.0,eccellente / restaurato,gallaratese,1,centralizzato,e
4,199000.0,38.0,1,0,1,fibra ottica | porta di sicurezza | esposizion...,4,no,0,80,1930.0,eccellente / restaurato,navigli - darsena,1,indipendente,g
5,299000.0,45.0,1,0,1,cancello elettrico | fibra ottica | porta di s...,5,no,0,100,1911.0,buone condizioni /vivibile,porta romana - medaglie d'oro,1,centralizzato,g
9,1290000.0,170.0,2,1,4,porta di sicurezza | terrazza | terrazza |port...,6,no,0,500,1970.0,buone condizioni /vivibile,de angeli,6,centralizzato,g


In [19]:
#Spese condominiali has been made numeric.
houses.loc[houses["spese.condominiali"] == "nessun costo condominiale", "spese.condominiali"] = 0
houses["spese.condominiali"] = pd.to_numeric(houses["spese.condominiali"])

In [25]:
#condizioni has been made made string.
houses["condizioni"] = houses["condizioni"].astype(str)

In [27]:
houses["zona"] = houses["zona"].astype(str)

In [29]:
houses["piano"] = houses["piano"].astype(str)

In [31]:
houses["riscaldamento.centralizzato"] = houses["riscaldamento.centralizzato"].astype(str)

In [32]:
houses["classe.di.efficienza.energetica"] = houses["classe.di.efficienza.energetica"].astype(str)

The following is dedicated to "altre.caratteristiche". Each characteristic has been made a specific feature of a house.

In [47]:
houses["altre.caratteristiche"].iloc[0].split("\s|\s", maxsplit=-1)

['fibra ottica',
 'videocitofono',
 "sistema d'allarme",
 'porta di sicurezza',
 'esposizione esterna',
 'guardaroba |portinaio mezza giornata',
 'sistema televisivo centralizzato',
 'parzialmente arredato|infissi esterni in doppio vetro / pvc']

In [55]:
def splitString (string, split):
    return re.split(split,string)

In [ ]:
re.split(r"\s*\|\s*",houses["altre.caratteristiche"].iloc[0])

['fibra ottica',
 'videocitofono',
 "sistema d'allarme",
 'porta di sicurezza',
 'esposizione esterna',
 'guardaroba',
 'portinaio mezza giornata',
 'sistema televisivo centralizzato',
 'parzialmente arredato',
 'infissi esterni in doppio vetro / pvc']

In [66]:
set(splitString(houses["altre.caratteristiche"].iloc[400],r"\s*\|\s*"))

{'cantina',
 'doppia esposizione',
 'fibra ottica',
 'guardaroba',
 'infissi esterni in doppio vetro / pvc',
 'parzialmente arredato',
 'porta di sicurezza',
 'sistema televisivo centralizzato',
 'terrazza',
 'videocitofono'}

In [70]:
#A cycle to get all the single other attributes of a house.
attributes = set()
for house in range(0,houses.shape[0]):
    attributes.update(set(splitString(houses["altre.caratteristiche"].iloc[house],r"\s*\|\s*")))

In [88]:
#Adding each characteristic as a new feature
for feature in attributes:
    houses[feature] = 0

In [90]:
#Each other characteritics: 0 if absent, 1 if present.
for house in range(0,houses.shape[0]):
    for feature in splitString(houses["altre.caratteristiche"].iloc[house],r"\s*\|\s*"):
        houses.iloc[house, houses.columns.get_loc(feature)] = 1

In [102]:
houses = houses.drop('altre.caratteristiche', axis=1)

In [103]:
#Make dummies.

,prezzo,metri.quadrati,numero.bagni,ascensore,numero.stanze,numero.totale.piani.edificio,parcheggio,disponibile,spese.condominiali,anno.di. Costruzione,...,guardaroba,portinaio tutto il giorno,infissi esterni in doppio vetro / pvcexposure south,taverna,esposizione esterna,terrazza,parzialmente arredato,private and giardino condiviso,giardino privato,parzialmente arredato
1,500000.0,57.0,1,1,2,7,1 in garage/box,0,0,2010.0,...,1,0,0,0,1,0,0,0,0,1
2,395000.0,92.0,1,1,3,8,no,0,200,1960.0,...,1,0,0,0,0,1,0,0,0,0
4,199000.0,38.0,1,0,1,4,no,0,80,1930.0,...,0,0,0,0,0,0,0,0,0,0
5,299000.0,45.0,1,0,1,5,no,0,100,1911.0,...,0,0,0,0,0,0,0,0,0,0
9,1290000.0,170.0,2,1,4,6,no,0,500,1970.0,...,0,0,0,0,0,1,0,0,0,0
